In [20]:
import pinecone
from google.cloud import bigquery
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import BigQueryLoader
from pydantic import BaseModel
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from dotenv import load_dotenv
import os

# Load the environment variables from the .env file
load_dotenv()

# Access environment variables
api_key = os.getenv("OPENAI_KEY")
pinecone_key = os.getenv('PINECONE_API_KEY')
pinecone_env = os.getenv('PINECONE_ENV')

In [21]:
embeddings = OpenAIEmbeddings(openai_api_key=api_key)

In [22]:

pinecone.init(api_key=pinecone_key,
              environment=pinecone_env)
# pinecone.create_index("chatbot", dimension=1536)

In [23]:
PROJECT = "wagon-bootcamp-377120"
DATASET = "g_adventures_dataset"
TABLE = "one_month"

In [24]:
found_itineraries = ['Morocco Kasbahs & Desert', 'Coastal Morocco: Waves & Market Stalls']

In [25]:
PROJECT = "wagon-bootcamp-377120"
DATASET = "g_adventures_dataset"
TABLE = "one_month"

query = f"""
    SELECT 
      DISTINCT tour_name
    FROM {PROJECT}.{DATASET}.{TABLE}
    """

client = bigquery.Client(project="wagon-bootcamp-377120")
query_job = client.query(query)
result = query_job.result()
df = result.to_dataframe()

In [26]:
tour_name_list = df['tour_name']

In [27]:
tour_name_list

0      Western and Central Europe: Venice, the Alps &...
1       Costa Rica: Volcanoes, Cloud Forests & Zip-Lines
2      Cancún Escape: Beaches, Cenotes, & Crystal-Cle...
3      Yucatan & Cancún: Merida, Mayan Ruins, & Beach...
4      Inca Adventure: Cusco, Machu Picchu & the Best...
                             ...                        
346                      Explore Kruger & Victoria Falls
347    Morocco Family Journey: Ancient Souks to the S...
348           Beijing to Tokyo: The Great Wall & Mt Fuji
349                                        Wellness Bali
350                                  Wellness Costa Rica
Name: tour_name, Length: 351, dtype: object

In [42]:
tools = []

for file in tour_name_list:
    
    pinecone_index = pinecone.Index("chatbot")
    vectorstore = Pinecone(pinecone_index, embeddings.embed_query, "text")
    
    filters = {"tour_name": file}

    # completion llm
    llm = ChatOpenAI(
        openai_api_key=api_key,
        model_name='gpt-3.5-turbo',
        temperature=0.0
    )

    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(search_kwargs={'filter': filters})
    )

    filters = {"tour_name": file}
    
    # Wrap retrievers in a Tool
    tools.append(
        Tool(
            name=file.replace(" ", "_").replace("&", "_").replace(":", "_").replace(",", "_").replace("__", "_"),
            description=f"useful when you want to answer questions about {file}",
            func=RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever(search_kwargs={'filter': filters})),
        )
    )

In [45]:
agent = initialize_agent(
    agent=AgentType.OPENAI_FUNCTIONS,
    tools=tools[10:20],
    llm=llm,
    verbose=True,
    max_iterations=5,
    early_stopping_method="generate",
)

In [46]:
%%time
response = agent({"input": "How fit do i need to be for the Kasbahs & Desert itinerary?"})
response['output']



> Entering new AgentExecutor chain...
To fully enjoy the Kasbahs & Desert itinerary, it is recommended to have a moderate level of fitness. This itinerary includes walking tours in the cities and exploring the desert landscapes, which may involve walking on uneven terrain and climbing sand dunes. Additionally, there may be some optional activities such as camel riding or hiking in the Atlas Mountains. While these activities are not extremely strenuous, having a reasonable level of fitness will enhance your experience.

> Finished chain.
CPU times: user 12.7 ms, sys: 247 µs, total: 12.9 ms
Wall time: 6.53 s


'To fully enjoy the Kasbahs & Desert itinerary, it is recommended to have a moderate level of fitness. This itinerary includes walking tours in the cities and exploring the desert landscapes, which may involve walking on uneven terrain and climbing sand dunes. Additionally, there may be some optional activities such as camel riding or hiking in the Atlas Mountains. While these activities are not extremely strenuous, having a reasonable level of fitness will enhance your experience.'